In [3]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer

In [4]:
cancer = load_breast_cancer()
cancer

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [5]:
import pandas as pd
import numpy as np
# 데이터셋 불러오기
from sklearn.datasets import load_breast_cancer
# 데이터 분리
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
# 모델 및 평가 지표
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
# 최적화 분포 정의 (RandomSearchCV용)
from scipy.stats import uniform, randint
import time

In [8]:
X, y = cancer.data, cancer.target

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [12]:
model = LogisticRegression(max_iter=1000, random_state=42)

In [13]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],     # 정규화 강도 (역수)
    'penalty': ['l1', 'l2'],                  # 정규화 방식
    'solver': ['liblinear', 'saga']           # 최적화 알고리즘
}

In [14]:
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,                    # 5-fold cross validation
    scoring='accuracy',      # 평가 지표
    n_jobs=-1,              # 병렬 처리 (모든 CPU 사용)
    verbose=2               # 진행 상황 출력
)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


,estimator,LogisticRegre...ndom_state=42)
,param_grid,"{'C': [0.001, 0.01, ...], 'penalty': ['l1', 'l2'], 'solver': ['liblinear', 'saga']}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l1'


In [15]:
# 5. 결과 확인
print(f"최적 파라미터: {grid_search.best_params_}")
print(f"최적 점수: {grid_search.best_score_:.3f}")

최적 파라미터: {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}
최적 점수: 0.967


In [16]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

In [17]:
y_pred

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0])

In [18]:
# 모든 결과를 DataFrame으로
import pandas as pd

results = pd.DataFrame(grid_search.cv_results_)

In [19]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004906,0.001206,0.003309,0.002628,0.001,l1,liblinear,"{'C': 0.001, 'penalty': 'l1', 'solver': 'libli...",0.967033,0.890110,0.956044,0.868132,0.890110,0.914286,0.039560,23
1,0.164095,0.012294,0.001600,0.000800,0.001,l1,saga,"{'C': 0.001, 'penalty': 'l1', 'solver': 'saga'}",0.967033,0.890110,0.956044,0.868132,0.879121,0.912088,0.041117,24
2,0.003005,0.001791,0.001102,0.000203,0.001,l2,liblinear,"{'C': 0.001, 'penalty': 'l2', 'solver': 'libli...",0.989011,0.868132,0.956044,0.879121,0.912088,0.920879,0.045786,9
3,0.125352,0.011273,0.001303,0.000874,0.001,l2,saga,"{'C': 0.001, 'penalty': 'l2', 'solver': 'saga'}",0.956044,0.879121,0.967033,0.868132,0.901099,0.914286,0.040166,19
4,0.006984,0.002112,0.000802,0.000401,0.010,l1,liblinear,"{'C': 0.01, 'penalty': 'l1', 'solver': 'liblin...",0.967033,0.868132,0.956044,0.868132,0.912088,0.914286,0.041931,19
5,0.172246,0.016511,0.000801,0.000749,0.010,l1,saga,"{'C': 0.01, 'penalty': 'l1', 'solver': 'saga'}",0.956044,0.879121,0.967033,0.868132,0.901099,0.914286,0.040166,19
6,0.002705,0.000401,0.000801,0.000748,0.010,l2,liblinear,"{'C': 0.01, 'penalty': 'l2', 'solver': 'liblin...",1.000000,0.879121,0.956044,0.890110,0.901099,0.925275,0.045786,8
7,0.121646,0.005746,0.001003,0.000634,0.010,l2,saga,"{'C': 0.01, 'penalty': 'l2', 'solver': 'saga'}",0.967033,0.868132,0.967033,0.868132,0.912088,0.916484,0.044284,11
8,0.089262,0.037800,0.000601,0.000491,0.100,l1,liblinear,"{'C': 0.1, 'penalty': 'l1', 'solver': 'libline...",0.978022,0.890110,0.956044,0.879121,0.901099,0.920879,0.038945,9
9,0.173234,0.015427,0.001002,0.000632,0.100,l1,saga,"{'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}",0.967033,0.868132,0.967033,0.868132,0.901099,0.914286,0.044719,19


In [20]:
# 주요 컬럼
results[['params', 'mean_test_score', 'rank_test_score']].head(10)

# 파라미터별 평균 점수
for param in param_grid.keys():
    print(f"{param} 효과:")    
    print(results.groupby(f'param_{param}')['mean_test_score'].mean())

# 시각화
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
pivot = results.pivot_table(
    values='mean_test_score',
    index='param_max_depth',
    columns='param_n_estimators'
)
plt.imshow(pivot, cmap='viridis')
plt.colorbar(label='Accuracy')
plt.xlabel('n_estimators')
plt.ylabel('max_depth')
plt.title('Grid Search Heatmap')
plt.show()

C 효과:
param_C
0.001      0.915385
0.010      0.917582
0.100      0.921429
1.000      0.934066
10.000     0.939560
100.000    0.940110
Name: mean_test_score, dtype: float64
penalty 효과:
param_penalty
l1    0.926740
l2    0.929304
Name: mean_test_score, dtype: float64
solver 효과:
param_solver
liblinear    0.940476
saga         0.915568
Name: mean_test_score, dtype: float64


KeyError: 'param_max_depth'

<Figure size 1000x600 with 0 Axes>

In [21]:
# 가상의 손실함수: y = (x-3)^2 + 2
# 최소점은 x=3 에 있음

x = 9        # 초기위치 (산꼭대기)
lr = 0.1     # 학습률 (보폭)
for i in range(10):
    grad = 2 * (x - 3)  # 미분: 기울기(경사)
    x = x - lr * grad   # 내려가기
    print(f"{i+1}회차 후 x={x:.4f}")


1회차 후 x=7.8000
2회차 후 x=6.8400
3회차 후 x=6.0720
4회차 후 x=5.4576
5회차 후 x=4.9661
6회차 후 x=4.5729
7회차 후 x=4.2583
8회차 후 x=4.0066
9회차 후 x=3.8053
10회차 후 x=3.6442


In [22]:
# 손실함수: y = (x-3)^2
# 학습률이 큰 경우와 작은 경우 비교

def gradient_descent(lr, steps):
    x = 9
    for i in range(steps):
        grad = 2 * (x - 3)
        x = x - lr * grad
        print(f"{i+1}회차 | x={x:.4f}")
    print()

print("=== 학습률 0.1 (적당) ===")
gradient_descent(lr=0.1, steps=10)

print("=== 학습률 1.0 (너무 큼) ===")
gradient_descent(lr=1.0, steps=10)


=== 학습률 0.1 (적당) ===
1회차 | x=7.8000
2회차 | x=6.8400
3회차 | x=6.0720
4회차 | x=5.4576
5회차 | x=4.9661
6회차 | x=4.5729
7회차 | x=4.2583
8회차 | x=4.0066
9회차 | x=3.8053
10회차 | x=3.6442

=== 학습률 1.0 (너무 큼) ===
1회차 | x=-3.0000
2회차 | x=9.0000
3회차 | x=-3.0000
4회차 | x=9.0000
5회차 | x=-3.0000
6회차 | x=9.0000
7회차 | x=-3.0000
8회차 | x=9.0000
9회차 | x=-3.0000
10회차 | x=9.0000



In [23]:
# 가짜 데이터 (x, y)
X = [1, 2, 3, 4, 5, 6]
y = [2, 4, 6, 8, 10, 12]  # y = 2x

# 초기값
w = 0.0
lr = 0.1
batch_size = 2
epochs = 3

for epoch in range(epochs):
    print(f"Epoch {epoch+1}")
    for i in range(0, len(X), batch_size):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]

        # 간단한 경사하강 (w 업데이트)
        grad = -2 * sum((y_b - w*x_b)*x_b for x_b, y_b in zip(X_batch, y_batch)) / len(X_batch)
        w = w - lr * grad
        print(f"  batch {i//batch_size+1} | w={w:.4f}")


Epoch 1
  batch 1 | w=1.0000
  batch 2 | w=3.5000
  batch 3 | w=-5.6500
Epoch 2
  batch 1 | w=-1.8250
  batch 2 | w=7.7375
  batch 3 | w=-27.2612
Epoch 3
  batch 1 | w=-12.6306
  batch 2 | w=23.9459
  batch 3 | w=-109.9243


In [25]:
X = [1, 2, 3, 4, 5, 6]
y = [2, 4, 6, 8, 10, 12]

w = 0.0           # 가중치 초기값
lr = 0.1          # 학습률 (보폭)
batch_size = 2    # 2개씩 묶어서 학습
epochs = 3        # 전체를 3번 반복

for epoch in range(epochs):
    print(f"\n📚 Epoch {epoch+1} 시작")
    for i in range(0, len(X), batch_size):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]

        # 1. 예측
        y_pred = [w * x for x in X_batch]

        # 2. 오차 계산 (간단한 평균제곱오차의 미분)
        grad = -2 * sum((y_true - y_pred_i) * x_i for x_i, y_true, y_pred_i in zip(X_batch, y_batch, y_pred)) / len(X_batch)

        # 3. 가중치 업데이트
        w = w - lr * grad

        print(f"  ▶ Batch {i//batch_size+1} | grad={grad:.4f}, w={w:.4f}")



📚 Epoch 1 시작
  ▶ Batch 1 | grad=-10.0000, w=1.0000
  ▶ Batch 2 | grad=-25.0000, w=3.5000
  ▶ Batch 3 | grad=91.5000, w=-5.6500

📚 Epoch 2 시작
  ▶ Batch 1 | grad=-38.2500, w=-1.8250
  ▶ Batch 2 | grad=-95.6250, w=7.7375
  ▶ Batch 3 | grad=349.9875, w=-27.2612

📚 Epoch 3 시작
  ▶ Batch 1 | grad=-146.3062, w=-12.6306
  ▶ Batch 2 | grad=-365.7656, w=23.9459
  ▶ Batch 3 | grad=1338.7022, w=-109.9243


In [26]:
w = 9.0         # 초기값
lr = 0.1        # 고정 학습률
for step in range(10):
    grad = 2 * (w - 3)    # 기울기
    w = w - lr * grad
    print(f"Step {step+1}: w={w:.4f}")


Step 1: w=7.8000
Step 2: w=6.8400
Step 3: w=6.0720
Step 4: w=5.4576
Step 5: w=4.9661
Step 6: w=4.5729
Step 7: w=4.2583
Step 8: w=4.0066
Step 9: w=3.8053
Step 10: w=3.6442


In [27]:
w = 9.0
v = 0.0           # 초기 속도
lr = 0.1
beta = 0.9        # 관성 계수

for step in range(10):
    grad = 2 * (w - 3)
    v = beta * v + (1 - beta) * grad
    w = w - lr * v
    print(f"Step {step+1}: w={w:.4f}, v={v:.4f}")


Step 1: w=8.8800, v=1.2000
Step 2: w=8.6544, v=2.2560
Step 3: w=8.3383, v=3.1613
Step 4: w=7.9470, v=3.9128
Step 5: w=7.4959, v=4.5109
Step 6: w=7.0000, v=4.9590
Step 7: w=6.4737, v=5.2631
Step 8: w=5.9305, v=5.4315
Step 9: w=5.3831, v=5.4745
Step 10: w=4.8427, v=5.4037


In [28]:
w = 9.0
m = 0
v = 0
lr = 0.1
beta1, beta2 = 0.9, 0.999
eps = 1e-8

for step in range(10):
    grad = 2 * (w - 3)

    # 1) 1차/2차 모멘텀 업데이트
    m = beta1 * m + (1 - beta1) * grad
    v = beta2 * v + (1 - beta2) * (grad ** 2)

    # 2) 보정된 모멘텀
    m_hat = m / (1 - beta1 ** (step + 1))
    v_hat = v / (1 - beta2 ** (step + 1))

    # 3) 파라미터 업데이트
    w = w - lr * m_hat / ((v_hat ** 0.5) + eps)
    print(f"Step {step+1}: w={w:.4f}")


Step 1: w=8.9000
Step 2: w=8.8000
Step 3: w=8.7002
Step 4: w=8.6004
Step 5: w=8.5008
Step 6: w=8.4014
Step 7: w=8.3022
Step 8: w=8.2032
Step 9: w=8.1045
Step 10: w=8.0062


In [30]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score, recall_score, f1_score
import numpy as np

# 1️⃣ 데이터 불러오기
data = load_breast_cancer()
X, y = data.data, data.target

# 2️⃣ 평가지표 설정
scoring = {
    "accuracy": make_scorer(accuracy_score),
    "recall": make_scorer(recall_score),
    "f1": make_scorer(f1_score)
}

# 3️⃣ 교차검증 세팅
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4️⃣ 평가 함수
def evaluate_model(model, name):
    pipe = Pipeline([("scaler", StandardScaler()), ("clf", model)])
    scores = cross_validate(pipe, X, y, cv=cv, scoring=scoring)
    print(f"\n📊 {name}")
    print("-----------------------------")
    for metric, vals in scores.items():
        if metric.startswith("test_"):
            print(f"{metric[5:]:>8}: {np.mean(vals):.4f} ± {np.std(vals):.4f}")

# 5️⃣ Optimizer 1 : SGD (순수 경사하강)
sgd = SGDClassifier(loss="log_loss", max_iter=1000, random_state=42)
evaluate_model(sgd, "SGD (순수 경사하강)")

# 6️⃣ Optimizer 2 : Momentum (MLP에서 구현)
mlp_momentum = MLPClassifier(hidden_layer_sizes=(16,), activation="relu",
                             solver="sgd", momentum=0.9, learning_rate_init=0.01,
                             max_iter=1000, random_state=42)
evaluate_model(mlp_momentum, "SGD + Momentum (MLP 기반)")

# 7️⃣ Optimizer 3 : Adam
mlp_adam = MLPClassifier(hidden_layer_sizes=(16,), activation="relu",
                         solver="adam", learning_rate_init=0.001,
                         max_iter=1000, random_state=42)
evaluate_model(mlp_adam, "Adam (MLP 기반)")



📊 SGD (순수 경사하강)
-----------------------------
accuracy: 0.9719 ± 0.0116
  recall: 0.9776 ± 0.0144
      f1: 0.9776 ± 0.0093

📊 SGD + Momentum (MLP 기반)
-----------------------------
accuracy: 0.9719 ± 0.0102
  recall: 0.9804 ± 0.0190
      f1: 0.9777 ± 0.0081

📊 Adam (MLP 기반)
-----------------------------
accuracy: 0.9667 ± 0.0187
  recall: 0.9804 ± 0.0143
      f1: 0.9737 ± 0.0144


In [6]:
import pandas as pd
import numpy as np
# 데이터셋 불러오기
from sklearn.datasets import load_breast_cancer
# 데이터 분리
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
# 모델 및 평가 지표
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
# 최적화 분포 정의 (RandomSearchCV용)
from scipy.stats import uniform, randint
import time

In [8]:
# 데이터 불러오기
data = load_breast_cancer()
X, y = data.data, data.target
# 데이터 분리 (훈련:테스트 = 8:2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# 참고: Stratify를 사용하여 y의 클래스 비율(악성/양성)을 훈련 세트와 테스트 세트가 동일하게 유지하도록 나눈다.

print(f"훈련 세트 크기: {X_train.shape}, 테스트 세트 크기: {X_test.shape}")

훈련 세트 크기: (455, 30), 테스트 세트 크기: (114, 30)


In [9]:
# 1. 모델 정의
# 초기 가중치를 동일하게 설정하여 비교의 공정성을 확보합니다.
# hidden_layer_sizes는 초보 단계에서는 간단하게 100으로 고정합니다.

# 순수 SGD (momentum=0.0)
sgd_model = MLPClassifier(
    hidden_layer_sizes=(100,), 
    solver='sgd', 
    learning_rate_init=0.01, 
    momentum=0.0, 
    max_iter=1000, 
    random_state=42
)

# Momentum (momentum=0.9)
momentum_model = MLPClassifier(
    hidden_layer_sizes=(100,), 
    solver='sgd', 
    learning_rate_init=0.01, 
    momentum=0.9, 
    max_iter=1000, 
    random_state=42
)

# Adam (solver='adam')
adam_model = MLPClassifier(
    hidden_layer_sizes=(100,), 
    solver='adam', 
    learning_rate_init=0.001, # Adam의 일반적인 추천 학습률 [cite: 158]
    max_iter=1000, 
    random_state=42
)

models = {
    "Pure SGD": sgd_model,
    "Momentum": momentum_model,
    "Adam": adam_model
}

results = []

# 2. 학습 및 평가
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] 

    # FN (False Negative) 계산 및 평가 지표 수집
    cm = confusion_matrix(y_test, y_pred)
    # TN: 0을 0으로, FP: 0을 1로, FN: 1을 0으로, TP: 1을 1로
    # 참고: 유방암 데이터셋에서 악성은 0, 양성은 1입니다. 
    # 따라서 악성(0)을 양성(1)으로 잘못 분류하는 것이 FN입니다.
    # 하지만 MLPClassifier의 confusion_matrix는 클래스 인덱스를 따르므로 
    # 클래스 0(악성)에 대한 FN을 찾아야 합니다.
    
    # 클래스 0 (악성)을 1 (양성)으로 예측한 경우 (FP)를 
    # **의료적 맥락**에서는 FN(악성인데 놓침)으로 간주합니다.
    # 여기서 악성(0)은 Negative, 양성(1)은 Positive입니다.
    
    # 헷갈리실 수 있으니, 의료 맥락의 FN(실제 악성인데 양성으로 예측)을 구하려면
    # 실제 악성(0)이면서 예측 양성(1)인 값, 즉 FP를 찾아서 FN_Count로 사용합니다.
    
    # cm: [[TN, FP], [FN, TP]]
    tn, fp, fn, tp = cm.ravel()
    
    results.append({
        "Optimizer": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "AUC": roc_auc_score(y_test, y_prob),
        # 의료 맥락의 FN (실제 악성을 놓친 수) = FP (실제 0을 1로 예측)
        "FN_Count (의료적 위험)": fp, 
        "TN": tn, "FP": fp, "FN": fn, "TP": tp
    })

results_df = pd.DataFrame(results).sort_values(by='FN_Count (의료적 위험)')

In [12]:
print("### 📊 Optimizer 비교 결과 (MLPClassifier 사용)")
print("FN_Count (의료적 위험): 실제 악성(0)을 양성(1)으로 잘못 예측한 수입니다. 이 값이 가장 작아야 합니다.\n")
results_df

### 📊 Optimizer 비교 결과 (MLPClassifier 사용)
FN_Count (의료적 위험): 실제 악성(0)을 양성(1)으로 잘못 예측한 수입니다. 이 값이 가장 작아야 합니다.



,Optimizer,Accuracy,AUC,FN_Count (의료적 위험),TN,FP,FN,TP
2,Adam,0.921053,0.982143,6,36,6,3,69
0,Pure SGD,0.517544,0.539683,19,23,19,36,36
1,Momentum,0.631579,0.500000,42,0,42,0,72
